### 회사의 영업 전환 비율

In [1]:
import pandas as pd

In [2]:
df_train = pd.read_csv("train.csv")
df_test = pd.read_csv("submission.csv")

In [106]:
# unique 값 확인
import numpy as np
result, a = 0,0
for col in np.sort(df_test['lead_owner'].unique()):
    if col not in np.sort(df_train['lead_owner'].unique()):
        print(col)
        num = len(df_test[df_test['lead_owner'] == col])
        a += 1
        result += num

127
135
136
308
313
315
316
320
323
327
331
336
347
353
356
373
377
379
427
554
558
562
565
571
573
576
577
581
587
588
591
595
723
727
731
733
734
735
736
747
748
749
750
751
752
766
769
772
773
777
821
828
844
850
866
867
868
874
892
907
908
911
922
960
962
963
965
966
969
978
999
1023
1024
1025
1026
1028
1032
1033
1046
1055
1056
1058
1062
1063
1108


In [4]:
lead_owner_dict ={}
col_name = 'customer_idx'
flag = df_train.groupby(col_name)['is_converted'].agg(['sum','count'])
for idx, plus, counts in zip(flag.index,flag['sum'],flag['count']):
    cond =  df_train[col_name] == idx
    ratio = (plus / counts) * 100
    df_train.loc[cond,f'{col_name}_converted_ratio'] = round(ratio,2)
    lead_owner_dict[idx] = ratio

In [7]:
df_train['customer_idx_converted_ratio'].value_counts().sort_index(ascending=False)

customer_idx_converted_ratio
100.00     4387
93.75        16
92.86        14
91.67        12
89.47        19
88.89         9
88.37        43
80.00        10
76.92        13
75.00         8
66.67        54
60.00        15
50.00       224
40.00        20
33.33       141
28.57        28
25.00        56
22.22         9
20.00        45
16.67         6
14.29        35
12.50         8
11.11         9
10.00        10
9.09         11
5.26         19
4.44         45
2.99         67
1.96         51
1.69         59
1.54         65
1.19         84
1.09        367
0.87       7686
0.62        162
0.30        331
0.00      45161
Name: count, dtype: int64

In [25]:
cus = df_train[df_train['customer_idx_converted_ratio']==40.00]

In [26]:
cus['bant_submit'].sum() / len(cus['bant_submit'])

0.5125

### customer_idx 범주화

In [52]:
frame = df_train['lead_owner_converted_ratio'].value_counts().sort_index(ascending=False).to_frame()

In [56]:
frame.reset_index(inplace = True)

In [124]:
frame2 = df_train.groupby('lead_owner_converted_ratio')['customer_idx'].unique().sort_index(ascending=False).reset_index()

In [131]:
for idx,value in enumerate(frame2['customer_idx']):
    frame2.loc[idx,'customer_idx_2'] = len(value)

In [125]:
bin = [0.0001,10.0001,20.0001,30.0001,40.0001,50.0001,60.0001,70.0001,80.0001,90.0001,100.00,100.001]
frame2['lead_owner_converted_ratio'] = pd.cut(frame2['lead_owner_converted_ratio'], bins=bin, right = False,labels= False)
frame2['lead_owner_converted_ratio'] = frame2['lead_owner_converted_ratio']+1
frame2['lead_owner_converted_ratio'].fillna(0.0,axis = 0, inplace  = True)

In [136]:
frame2.groupby('lead_owner_converted_ratio')['customer_idx_2'].sum()

lead_owner_converted_ratio
0.0     15293.0
1.0     14697.0
2.0      3881.0
3.0      2062.0
4.0      1013.0
5.0       626.0
6.0       360.0
7.0       280.0
8.0       156.0
9.0        35.0
10.0      150.0
11.0      154.0
Name: customer_idx_2, dtype: float64

In [161]:
customer_idx_group = df_train.groupby('customer_idx')['customer_idx_converted_ratio'].mean().reset_index()
customer_idx_dict = dict(zip(customer_idx_group['customer_idx'], customer_idx_group['customer_idx_converted_ratio']))

In [165]:
customer_idx_dict[23813]

100.0

In [168]:
import numpy as np
for i in frame2['lead_owner_converted_ratio'].unique():
    cond = frame2['lead_owner_converted_ratio'] == i
    arr = frame2[cond]['customer_idx'].to_list()
    arr = np.concatenate(arr).tolist()
    result = []
    for idx in arr:
        value = customer_idx_dict[idx]
        result.append(value)
    frame2.loc[cond,'ratio_mean'] = np.mean(result)
            

In [169]:
frame2

,lead_owner_converted_ratio,customer_idx,customer_idx_2,ratio_mean
0,11.0,"[23813, 11012, 20371, 3414, 3576, 3580, 47466,...",154.0,92.837468
1,10.0,"[14746, 20943, 5101, 16444, 45319, 10350, 9324...",17.0,90.759000
2,10.0,"[6188, 33657, 43611, 9451, 47466, 43604, 10970...",18.0,90.759000
3,10.0,"[6480, 1467, 26966, 27458, 7774, 46290, 6479, ...",84.0,90.759000
4,10.0,"[7486, 8017, 17814, 17812, 11613, 7345, 7344, ...",17.0,90.759000
...,...,...,...,...
249,1.0,"[40382, 6795, 26199, 17222, 37488, 17920, 1544...",293.0,1.611942
250,1.0,"[27153, 22103, 26820, 29197, 27933, 2862, 4746...",48.0,1.611942
251,1.0,"[16351, 7930, 8894, 31911, 30136, 38909, 14626...",317.0,1.611942
252,1.0,"[15894, 27850, 39235, 39349, 35180, 21607, 156...",561.0,1.611942


In [185]:
lead_owner_customer_idx_mean=frame2.groupby('lead_owner_converted_ratio')['ratio_mean'].mean().to_dict()

In [192]:
cd ..

/Users/goodyoung/Desktop/GIt/LG_Aimers


In [193]:
from phase2_module.src.utils.manage_pkl_files import PickleManager as pkl_manager

In [194]:
pkl_manager.save("customer_idx","lead_owner_customer_idx_mean",lead_owner_customer_idx_mean)

In [186]:
lead_owner_customer_idx_mean

{0.0: 0.19189629242136927,
 1.0: 1.6119418929033138,
 2.0: 7.668162844627673,
 3.0: 12.45754122211445,
 4.0: 26.481826258637707,
 5.0: 30.926277955271562,
 6.0: 31.139527777777776,
 7.0: 52.375428571428564,
 8.0: 46.288717948717945,
 9.0: 65.0217142857143,
 10.0: 90.75899999999999,
 11.0: 92.83746753246754}

In [ ]:
for idx,value in enumerate(frame2['customer_idx']):
    frame2.loc[idx,'customer_idx_2'] = len(value)

In [ ]:
lead_owner 다른게 몇개 이상

In [178]:
df_train[df_train['lead_owner']==16][['customer_idx']].value_counts()

customer_idx
47466           69
26337           13
33776            9
46271            7
33763            7
                ..
17327            1
17328            1
17534            1
17535            1
25075            1
Name: count, Length: 255, dtype: int64

In [172]:
df_train[['customer_idx','customer_idx_converted_ratio']]

,customer_idx,customer_idx_converted_ratio
0,32160,50.00
1,23122,100.00
2,1755,66.67
3,4919,100.00
4,17126,100.00
...,...,...
59294,33747,0.00
59295,35420,0.00
59296,19249,0.00
59297,40327,0.00


In [170]:
df_train[['lead_owner','lead_owner_converted_ratio']]

,lead_owner,lead_owner_converted_ratio
0,0,4.66
1,1,75.00
2,2,32.50
3,3,37.93
4,4,78.95
...,...,...
59294,694,0.00
59295,39,0.00
59296,125,0.00
59297,134,0.00


In [179]:
df_train['lead_owner_converted_ratio'].value_counts().sort_index(ascending=False)

lead_owner_converted_ratio
100.00      343
98.88        89
96.43        28
95.74       188
95.24        21
          ...  
0.61        328
0.41        487
0.31        326
0.25       1215
0.00      24237
Name: count, Length: 254, dtype: int64

In [154]:
df_train['customer_converted_ratio'].value_counts().sort_index(ascending=False)

customer_converted_ratio
100.00     4387
93.75        16
92.86        14
91.67        12
89.47        19
88.89         9
88.37        43
80.00        10
76.92        13
75.00         8
66.67        54
60.00        15
50.00       224
40.00        20
33.33       141
28.57        28
25.00        56
22.22         9
20.00        45
16.67         6
14.29        35
12.50         8
11.11         9
10.00        10
9.09         11
5.26         19
4.44         45
2.99         67
1.96         51
1.69         59
1.54         65
1.19         84
1.09        367
0.87       7686
0.62        162
0.30        331
0.00      45161
Name: count, dtype: int64

In [155]:
df_train[df_train['customer_converted_ratio'] == 92.86]

,bant_submit,customer_country,business_unit,com_reg_ver_win_rate,customer_idx,customer_type,enterprise,historical_existing_cnt,id_strategic_ver,it_strategic_ver,...,ver_pro,ver_win_rate_x,ver_win_ratio_per_bu,business_area,business_subarea,lead_owner,is_converted,customer_converted_ratio,lead_owner_converted_ratio,customer_idx_converted_ratio
13550,0.5,/malabon/Philippines,AS,NaN,26337,Installer,Enterprise,0.0,NaN,NaN,...,0,NaN,NaN,NaN,NaN,16,True,92.86,7.99,92.86
13551,0.5,/Makati/Philippines,AS,NaN,26337,End-Customer,Enterprise,0.0,NaN,NaN,...,0,NaN,NaN,NaN,NaN,16,True,92.86,7.99,92.86
13554,0.5,/manila/Philippines,AS,NaN,26337,Specifier/ Influencer,Enterprise,0.0,NaN,NaN,...,0,NaN,NaN,NaN,NaN,16,True,92.86,7.99,92.86
13560,0.5,/CALOOCAN CITY/Philippines,AS,NaN,26337,End-Customer,Enterprise,0.0,NaN,NaN,...,0,NaN,NaN,NaN,NaN,16,True,92.86,7.99,92.86
13563,0.5,/Navotas/Philippines,AS,NaN,26337,End-Customer,Enterprise,0.0,NaN,NaN,...,0,NaN,NaN,NaN,NaN,16,True,92.86,7.99,92.86
13577,0.5,/Quezon City/Philippines,AS,NaN,26337,End-Customer,Enterprise,0.0,NaN,NaN,...,0,NaN,NaN,NaN,NaN,16,True,92.86,7.99,92.86
13578,0.5,/quezon city/Philippines,AS,NaN,26337,End-Customer,Enterprise,0.0,NaN,NaN,...,0,NaN,NaN,NaN,NaN,16,True,92.86,7.99,92.86
13587,0.5,/Quezon City/Philippines,AS,NaN,26337,Others,Enterprise,0.0,NaN,NaN,...,0,NaN,NaN,NaN,NaN,16,True,92.86,7.99,92.86
13591,0.5,/BULACAN/Philippines,AS,NaN,26337,Homeowner,Enterprise,0.0,NaN,NaN,...,0,NaN,NaN,NaN,NaN,16,True,92.86,7.99,92.86
13592,0.5,"/Rodriguez, Rizal/Philippines",AS,NaN,26337,End-Customer,Enterprise,0.0,NaN,NaN,...,0,NaN,NaN,NaN,NaN,16,True,92.86,7.99,92.86


In [147]:
df_train[df_train['lead_owner_converted_ratio'] == 100.00]['bant_submit'].value_counts()

bant_submit
1.00    137
0.75    102
0.50     72
0.25     26
0.00      6
Name: count, dtype: int64

In [ ]:
class Convert_ratio:
    def __init__(self, **kwargs):
        for key, value in kwargs.items():
            setattr(self, key, value)
    def make_ratio_dict(self,df,col_name):
        """비율 딕셔너리 저장"""
        from phase2_module.src.utils.manage_pkl_files import PickleManager as pkl_manager
        dict_ ={}
        flag = df.groupby(col_name)['is_converted'].agg(['sum','count'])
        for idx, plus, counts in zip(flag.index,flag['sum'],flag['count']):
            cond =  df[col_name] == idx
            ratio = (plus / counts) * 100
            df.loc[cond,f'{col_name}_converted_ratio'] = round(ratio,2)
            dict_[idx] = ratio
        # save
        pkl_manager.save(f"{col_name}",f"{col_name}_converted_ratio",dict_)
        
    def ratio_preprocessing(self,df, col_name, ratio_dict):
        # 검증
        for i in np.sort(df[col_name].unique()):
            if i not in ratio_dict.keys(): # 없으면 추가
                self.lead_owner_converted_ratio[i] = 0.0
        df[f'{col_name}_converted_ratio'] = df[col_name].map(ratio_dict)
        return df
        
    def lead_owner(self,df):
        return self.ratio_preprocessing(df, 'lead_owner', self.lead_owner_converted_ratio)
    
    def apply(self, df, module_list):
        if not module_list:
            raise ValueError("Not used modules")

        if not isinstance(module_list, list):
            module_list = [module_list]

        for module in module_list:
            method = getattr(self, module)

            df = method(df)

        return df

In [218]:
cd ../../..

/Users/goodyoung/Desktop/GIt/LG_Aimers


In [221]:
from phase2_module.src.utils.manage_pkl_files import PickleManager as pkl_manager

In [222]:
pkl_manager.save("lead_owner","lead_owner_converted_ratio",lead_owner_dict)

## 파생변수 결측치 채우기

- histo : 이전에 Converted(영업 전환) 되었던 횟수
- idit: id_strategic_ver이나 it_strategic_ver 값 중 하나라도 1의 값을 가지면 1 값으로 표현
-  com_reg_ver_win_rate: Vertical Level 1, business unit, region을 기준으로 oppty 비율을 계산
-  ver_win_rate: 전체 Lead 중에서 Vertical을 기준으로 Vertical 수 비율과 Vertical 별 Lead 수 대비 영업 전환 성공 비율 값을 곱한 값
-  ver_win_ratio 특정 Vertical Level1의 Business Unit 별 샘플 수 대비 영업 전환된 샘플 수의 비율을 계산
-  business_area: 고객의 사업 영역
-  business_area: 고객의 세부 사업 영역

In [156]:
df = df_train[['com_reg_ver_win_rate','historical_existing_cnt','idit_strategic_ver','ver_win_rate_x','ver_win_ratio_per_bu'
,'business_area','business_subarea']]

In [159]:
df_train

,bant_submit,customer_country,business_unit,com_reg_ver_win_rate,customer_idx,customer_type,enterprise,historical_existing_cnt,id_strategic_ver,it_strategic_ver,...,ver_pro,ver_win_rate_x,ver_win_ratio_per_bu,business_area,business_subarea,lead_owner,is_converted,customer_converted_ratio,lead_owner_converted_ratio,customer_idx_converted_ratio
0,1.00,/Quezon City/Philippines,AS,0.066667,32160,End-Customer,Enterprise,NaN,NaN,NaN,...,0,0.003079,0.026846,corporate / office,Engineering,0,True,50.00,4.66,50.00
1,1.00,/PH-00/Philippines,AS,0.066667,23122,End-Customer,Enterprise,12.0,NaN,NaN,...,0,0.003079,0.026846,corporate / office,Advertising,1,True,100.00,75.00,100.00
2,1.00,/Kolkata /India,AS,0.088889,1755,End-Customer,Enterprise,144.0,NaN,NaN,...,0,0.003079,0.026846,corporate / office,Construction,2,True,66.67,32.50,66.67
3,1.00,/Bhubaneswar/India,AS,0.088889,4919,End-Customer,Enterprise,NaN,NaN,NaN,...,0,0.003079,0.026846,corporate / office,IT/Software,3,True,100.00,37.93,100.00
4,1.00,/Hyderabad/India,AS,0.088889,17126,Specifier/ Influencer,Enterprise,NaN,NaN,NaN,...,0,0.003079,0.026846,corporate / office,NaN,4,True,100.00,78.95,100.00
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
59294,1.00,/Sląskie/Poland,AS,NaN,33747,End Customer,SMB,NaN,NaN,NaN,...,0,0.000026,0.028777,public facility,Others,694,False,0.00,0.00,0.00
59295,0.75,/Bogotá DC /Colombia,AS,0.040000,35420,Specifier/ Influencer,Enterprise,NaN,NaN,NaN,...,0,0.000026,0.028777,public facility,NaN,39,False,0.00,0.00,0.00
59296,0.75,/Pisco/Peru,AS,0.040000,19249,Specifier/ Influencer,Enterprise,NaN,NaN,NaN,...,0,0.000026,0.028777,public facility,NaN,125,False,0.00,0.00,0.00
59297,1.00,/santa cruz bolivia/Peru,AS,0.040000,40327,NaN,Enterprise,NaN,NaN,NaN,...,0,0.000026,0.028777,public facility,NaN,134,False,0.00,0.00,0.00


In [161]:
df['historical_existing_cnt'].value_counts()

historical_existing_cnt
4.0       2957
0.0       1973
1.0       1967
19.0      1441
16.0      1132
          ... 
404.0        1
145.0        1
1394.0       1
199.0        1
108.0        1
Name: count, Length: 136, dtype: int64

In [17]:
df_train.head()

,com_reg_ver_win_rate,historical_existing_cnt,idit_strategic_ver,ver_win_rate_x,ver_win_ratio_per_bu,business_area,business_subarea
0,0.066667,NaN,NaN,0.003079,0.026846,corporate / office,Engineering
1,0.066667,12.0,NaN,0.003079,0.026846,corporate / office,Advertising
2,0.088889,144.0,NaN,0.003079,0.026846,corporate / office,Construction
3,0.088889,NaN,NaN,0.003079,0.026846,corporate / office,IT/Software
4,0.088889,NaN,NaN,0.003079,0.026846,corporate / office,NaN


In [ ]:
df_